In [1]:
import pdal
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from app_logger import App_Logger
from files_loader import FileHandler

In [2]:
class Fetch3depData:
    '''
    This is a class fof fetching 3DEP data
    '''

    def __init__(self, public_data_url = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/", pipeline_json_path="fetch.json") -> None:
        self.logger = App_Logger().get_logger(__name__)
        self.file_handler = FileHandler()
        self.pipeline_json = self.file_handler.read_json(pipeline_json_path)
        self.public_data_url = public_data_url
        self.input_epsg = 3857
    def get_polygon_boundaries(self, polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=self.output_epsg, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=self.input_epsg)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        print(polygon_input)
        print(f"({[minx, maxx]},{[miny,maxy]})")

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    def get_pipeline(self, region: str, polygon: Polygon, output_filename: str = "temp"):
        boundaries, polygon_input = self.get_polygon_boundaries(polygon)

        full_dataset_path = f"{self.public_data_url}{region}/ept.json"

        self.pipeline_json['pipeline'][0]['filename'] = full_dataset_path
        self.pipeline_json['pipeline'][0]['bounds'] = boundaries
        self.pipeline_json['pipeline'][1]['polygon'] = polygon_input
        self.pipeline_json['pipeline'][3]['out_srs'] = f'EPSG:{self.output_epsg}'
        self.pipeline_json['pipeline'][4]['filename'] = "../data/laz/" + output_filename + ".laz"
        self.pipeline_json['pipeline'][5]['filename'] = "../data/tif/" + output_filename + ".tif"

        pipeline = pdal.Pipeline(json.dumps(self.pipeline_json))

        return pipeline
    def run_pipeline(self, polygon: Polygon, epsg, region: str = "IA_FullState"):
        self.output_epsg = epsg
        pipeline = self.get_pipeline(region, polygon)

        try:
            pipeline.execute()
            self.logger.info(f'Pipeline executed successfully.')
            return pipeline
        except RuntimeError as e:
            self.logger.exception('Pipeline execution failed')
            print(e)
    def make_geo_df(self, arr):
        geometry_points = [Point(x, y) for x, y in zip(arr["X"], arr["Y"])]
        elevetions = arr["Z"]
        df = gpd.GeoDataFrame(columns=["elevation", "geometry"])
        df['elevation'] = elevetions
        df['geometry'] = geometry_points
        df = df.set_geometry("geometry")
        df.set_crs(self.output_epsg, inplace=True)
        return df
    def get_data(self, polygon: Polygon, epsg):
        pipeline = self.run_pipeline(polygon, epsg)
        arr = pipeline.arrays[0]
        return self.make_geo_df(arr)

   

In [3]:
if(__name__ == '__main__'):
    MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.756055, 41.918115]
    polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))
    data_fetcher = Fetch3depData()
    print(data_fetcher.get_data(polygon, epsg=4326))

POLYGON((-10436887.43333523 5148706.389047224, -10436887.43333523 5148721.349314565, -10436876.301386151 5148721.349314565, -10436876.301386151 5148706.389047224, -10436887.43333523 5148706.389047224))
([-10436887.43333523, -10436876.301386151],[5148706.389047224, 5148721.349314565])


24-06-2022 15:06:35 - [ERROR] -  __main__ - (<ipython-input-2-4b0165ac5391>).run_pipeline(line 54) - Pipeline execution failed
Traceback (most recent call last):
  File "<ipython-input-2-4b0165ac5391>", line 50, in run_pipeline
    pipeline.execute()
  File "/home/user/anaconda3/envs/agritech/lib/python3.6/site-packages/pdal/pipeline.py", line 40, in execute
    return self.p.execute()
  File "libpdalpython.pyx", line 180, in pdal.libpdalpython.PyPipeline.execute
RuntimeError: filters.range: Unexpected argument 'polygon'.


filters.range: Unexpected argument 'polygon'.


AttributeError: 'NoneType' object has no attribute 'arrays'

In [9]:
def plot_raster(self,rast_data, title='', figsize=(10,10)):
        """
        Plots population count in log scale(+1)
        args:
            rast_data (np arrray): an array of the raster image
            title (str): the title of the image
            figsize (tuple): scale of the image to be displayed
        returns:
            pyplot image
        """
        plt.figure(figsize = figsize)
        im1 = plt.imshow(np.log1p(rast_data),) # vmin=0, vmax=2.1)

        plt.title("{}".format(title), fontdict = {'fontsize': 20})  
        plt.axis('off')
        plt.colorbar(im1, fraction=0.03)

In [10]:
def show_raster(path_to_raster):
    """
    displays a raster from a .tif raster file
    args:
        path_to_raster (str): path to the raster file
    returns:
        rasterio image
    """
    src = rasterio.open(path_to_raster)
    fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))
    show((src), cmap='Greys_r', contour=True, ax=axrgb)
    show_hist(src, bins=50, histtype='stepfilled',
            lw=0.0, stacked=False, alpha=0.3, ax=axhist)
    plt.show()

In [11]:
show_raster('../data/iowa.tif')


NameError: name 'rasterio' is not defined